<a href="https://colab.research.google.com/github/keonju2/2022_DL_project/blob/main/%EA%B1%B4%EC%A3%BC/yolov5_jupyterlab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt  # install
!pip install -q roboflow

Cloning into 'yolov5'...
remote: Enumerating objects: 12093, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 12093 (delta 23), reused 15 (delta 7), pack-reused 12047
Receiving objects: 100% (12093/12093), 11.90 MiB | 9.96 MiB/s, done.
Resolving deltas: 100% (8379/8379), done.
/home/jupyter/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 27.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 28.5 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.39.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.5.1 which is incompatible.
apache-beam 2.39.0 requires httplib2<0.20.0,>=0.8, but you have httplib2 0.20.4 which is

In [ ]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.11.0+cu102 (Tesla V100-SXM2-16GB)


In [ ]:
print(torch.cuda.device_count()) # 3

2


In [ ]:
!python train.py

train: weights=yolov5s.pt, cfg=, data=data/coco128.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-234-gd490bdd Python-3.7.12 torch-1.11.0+cu102 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, 

In [ ]:
from zipfile import ZipFile
import os

def unzip(source_file, dest_path):
    with ZipFile(source_file, 'r') as zf:
        zipInfo = zf.infolist()
        for member in zipInfo:
            try:
                member.filename = member.filename.encode('cp437').decode('euc-kr', 'ignore')
                zf.extract(member, dest_path)
            except:
                raise Exception('what?!')

In [ ]:
import os
os.getcwd()

In [ ]:
from google.cloud import storage

In [ ]:
file_names = '/home/jupyter/bb.zip'
unzip(file_names,'custom_dataset/val/labels')

In [ ]:
shutil.move('custom_dataset/val/labels/bb/자동차전용도로',
            'custom_dataset/val/labels')

'custom_dataset/val/labels/자동차전용도로'

In [ ]:
shutil.move('custom_dataset/val/labels/bb/도심로',
            'custom_dataset/val/labels')

'custom_dataset/val/labels/도심로'

In [ ]:
shutil.rmtree('custom_dataset/val/labels/bb')

In [ ]:
file_names = '/home/jupyter/labels.zip'
unzip(file_names,'custom_dataset/train')

In [ ]:
shutil.move('custom_dataset/train/labels/labels',
            'custom_dataset/')

'custom_dataset/labels'

In [ ]:
import os
os.remove('/home/jupyter/bb.zip')

In [ ]:
os.remove('/home/jupyter/labels.zip')

In [ ]:
client = storage.Client()
print("Client created using default project: {}".format(client.project))

Client created using default project: focal-cooler-343812


In [ ]:
buckets = client.list_buckets()

print("Buckets in {}:".format(client.project))
for item in buckets:
    print("\t" + item.name)

Buckets in focal-cooler-343812:
	cloud-ai-platform-b401b12d-7135-4e40-822e-f76792602368


In [ ]:
bucket_name = 'cloud-ai-platform-b401b12d-7135-4e40-822e-f76792602368'

bucket = client.get_bucket(bucket_name)

print("Bucket name: {}".format(bucket.name))
print("Bucket location: {}".format(bucket.location))
print("Bucket storage class: {}".format(bucket.storage_class))

Bucket name: cloud-ai-platform-b401b12d-7135-4e40-822e-f76792602368
Bucket location: US-CENTRAL1
Bucket storage class: REGIONAL


In [ ]:
bucket_name = 'cloud-ai-platform-b401b12d-7135-4e40-822e-f76792602368'
dl_dir = '/home/jupyter/'
prefix = 'gs://cloud-ai-platform-b401b12d-7135-4e40-822e-f76792602368/'

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)
print(bucket)
blobs = bucket.list_blobs()  # Get list of files
for blob in blobs:
    filename = blob.name.replace('/', '_') 
    blob.download_to_filename(dl_dir + filename)

<Bucket: cloud-ai-platform-b401b12d-7135-4e40-822e-f76792602368>


In [ ]:
file_names = '/home/jupyter/[원천]도심로.zip'
unzip(file_names,'custom_dataset/train/images')

In [ ]:
file_names = '/home/jupyter/[원천]자동차전용도로.zip'
unzip(file_names,'custom_dataset/train/images')

In [ ]:
file_names = '/home/jupyter/원천데이터-001.zip'
unzip(file_names,'custom_dataset/val/images')

In [ ]:
os.remove('/home/jupyter/[원천]도심로.zip')
os.remove('/home/jupyter/[원천]자동차전용도로.zip')
os.remove('/home/jupyter/원천데이터-001.zip')

In [ ]:
shutil.move('custom_dataset/val/images/bb',
            'custom_dataset/')

'custom_dataset/bb'

In [ ]:
shutil.rmtree('custom_dataset/val/images')

In [ ]:
shutil.move('custom_dataset/bb',
            'custom_dataset/val/images')

'custom_dataset/val/images'

In [ ]:
import yaml

make_yaml = {
             "train" : "/home/jupyter/yolov5/custom_dataset/train",
            "val": "/home/jupyter/yolov5/custom_dataset/val",

             "nc" : 32,
             "names": ['Vehicle_Car', 'Vehicle_Bus', 'Vehicle_Motorcycle', 'Vehicle_Unknown', 
                       'Pedestrian_Pedestrian', 'Pedestrian_Bicycle', 'Lane_White_Dash', 'Lane_White_Solid', 
                       'Lane_Yellow_Dash', 'Lane_Yellow_Solid', 'Lane_Blue_Dash', 'Lane_Blue_Solid', 
                       'TrafficLight_Red', 'TrafficLight_Yellow', 'TrafficLight_Green', 'TrafficLight_Arrow', 
                       'TrafficLight_RedArrow', 'TrafficLight_YellowArrow', 'TrafficLight_GreenArrow', 
                       'TrafficSign_Speed', 'TrafficSign_Else', 'RoadMark_StopLine', 'RoadMark_Crosswalk', 
                       'RoadMark_Number', 'RoadMark_Character', 'RoadMarkArrow_Straight', 'RoadMarkArrow_Left', 
                       'RoadMarkArrow_Right', 'RoadMarkArrow_StraightLeft', 'RoadMarkArrow_StraightRight', 
                       'RoadMarkArrow_Uturn', 'RoadMarkArrow_Else']
}
with open('data/custom_dataset.yaml', 'w') as outfile:
    yaml.dump(make_yaml, outfile, default_flow_style=False)

In [ ]:
lab =['Vehicle_Car', 'Vehicle_Bus', 'Vehicle_Motorcycle', 'Vehicle_Unknown', 
                       'Pedestrian_Pedestrian', 'Pedestrian_Bicycle', 'Lane_White_Dash', 'Lane_White_Solid', 
                       'Lane_Yellow_Dash', 'Lane_Yellow_Solid', 'Lane_Blue_Dash', 'Lane_Blue_Solid', 
                       'TrafficLight_Red', 'TrafficLight_Yellow', 'TrafficLight_Green', 'TrafficLight_Arrow', 
                       'TrafficLight_RedArrow', 'TrafficLight_YellowArrow', 'TrafficLight_GreenArrow', 
                       'TrafficSign_Speed', 'TrafficSign_Else', 'RoadMark_StopLine', 'RoadMark_Crosswalk', 
                       'RoadMark_Number', 'RoadMark_Character', 'RoadMarkArrow_Straight', 'RoadMarkArrow_Left', 
                       'RoadMarkArrow_Right', 'RoadMarkArrow_StraightLeft', 'RoadMarkArrow_StraightRight', 
                       'RoadMarkArrow_Uturn', 'RoadMarkArrow_Else']

In [ ]:
for (root, directories, files) in os.walk('/home/jupyter/yolov5/custom_dataset'):
    for file in files:
        file_path = os.path.join(root, file)
        change_file_path = file.replace('_v001_1','')
        file_path2 = os.path.join(root, change_file_path)
        os.rename(file_path,file_path2)

In [ ]:
for (root, directories, files) in os.walk('/home/jupyter/yolov5/custom_dataset'):
    for file in directories:
        file_path = os.path.join(root, file)
        if 'annotations' in file_path:
            change_file_path = file_path.replace('_annotations_v001_1','')
          
            shutil.move(file_path,change_file_path)

In [ ]:
!python -m torch.distributed.run --nproc_per_node 2 train.py --img 640 --batch 140 --epochs 30 --data data/custom_dataset.yaml --weights yolov5s.pt --device 0,1

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
train: weights=yolov5s.pt, cfg=, data=data/custom_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=140, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=0,1, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 1 commit. Use `git pull` or `git clone https://github.com/ultralytics/yolo

In [ ]:
import shutil

shutil.rmtree('custom_dataset/val/images/자동차전용도로')
shutil.rmtree('custom_dataset/val/labels/자동차전용도로')

In [ ]:
import os
import zipfile
 
fantasy_zip = zipfile.ZipFile('C:\\Stories\\Fantasy\\archive.zip', 'w')
 
for folder, subfolders, files in os.walk('C:\\Stories\\Fantasy'):
 
    for file in files:
        if file.endswith('.pdf'):
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), 'C:\\Stories\\Fantasy'), compress_type = zipfile.ZIP_DEFLATED)
 
fantasy_zip.close()
